In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.3.12")
@file:DependsOn("io.ktor:ktor-client-cio:2.3.12")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.3.12")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.3.12")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-14.2")
user

UserTO(id=8811c104-0826-4498-8d75-423e093aa824, username=Johann-14.2)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=6162c43f-4dfd-48e0-abc3-a6ff4f806940, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=1020356c-608c-4da9-880c-75529b1eee16, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=250501e8-d976-440b-819d-ab08cc3947c5, name=ING RON, unit=Currency(value=RON))
AccountTO(id=cbd97057-6d91-4564-8445-8f64a42b0eac, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=79532b89-6e23-4280-8f4c-80afaddb2d41, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=d0f95204-803c-4b51-8db3-c9c089868fbb, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=a9ddd90d-b8e0-4f8f-9e2c-266ae4191b13, name=Other, unit=Currency(value=RON))
AccountTO(id=546b8c6c-f654-4960-8ea5-83815ebbc8e6, name=BT RON, unit=Currency(value=RON))
AccountTO(id=63d6e5a8-181d-4801-a91d-527e29dd56a4, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=f2bfba9a-b5fa-4d25-8f8a-42592930a8e5, name=Expenses)
FundTO(id=6d1cbcb4-92a5-4ecf-9d5c-6e8393992575, name=Work Income)
FundTO(id=ff4f3cb9-3d65-4eec-9d8c-7f1674ddaa27, name=Savings)
FundTO(id=66ccb12f-25d1-435d-babf-2d5d500a7bde, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=9d564d62-5b0a-470d-8656-f8bbb11c8d72, status=COMPLETED, reason=null)